# RESNET50 Image Classification Model Acceleration

In [45]:
%pip install torch torchvision 
%pip install fiftyone

In [46]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchvision.io import decode_image 

from torchvision.models import resnet50, ResNet50_Weights 

import torch
import os
import time

### check whether GPU is available or not: 

In [47]:
print("GPU is available" if torch.cuda.is_available() else "GPU is not available")

GPU is available


### define the dataset class for our inference images:

In [48]:
class InferenceDataset(Dataset):
    def __init__(self, images_dir: str, transform = None): 
        self.images_dir = images_dir
        self.transform = transform
        self.images = sorted([img for img in os.listdir(self.images_dir) 
                              if img.lower().endswith((".jpg", ".jpeg", ".png"))
                              ])
    
    def __getitem__(self, idx): 
        img_path = os.path.join(self.images_dir, self.images[idx])
        img = decode_image(img_path)
        
        if self.transform: 
            img = self.transform(img).unsqueeze(0)
        
        return img

### define the dataloader

In [49]:
def data_loader(dataset, batch_size, num_workers, pin_memory):
    return DataLoader(
        dataset,
        batch_size=batch_size,        
        shuffle=False, # no shuffling to keep the order of images processing identical for fair comparison
        num_workers=num_workers,        
        pin_memory=pin_memory   
        )

### define `gpu_inference` function for GPU

In [50]:
def gpu_inference(model, loader, non_blocking = True): 
    device = torch.device('cuda')
    #move the model to gpu 
    model = model.to(device)
    #set to evaluation mode, this disables dropouts and batch normalizations
    model.eval()
     
    torch.cuda.synchronize()
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)

    start.record()

    with torch.no_grad():
        for x in loader:
            x = x.to(device, non_blocking=non_blocking)
            _ = model(x)

    end.record()
    torch.cuda.synchronize()

    gpu_time = start.elapsed_time(end)
    print("GPU time (ms):", gpu_time)
    return gpu_time



### define `cpu_inference` function for CPU

In [51]:
def cpu_inference(model, loader): 
    device = torch.device("cpu")
    model = model.to(device)
    model.eval()

    start = time.time()
    with torch.no_grad():
        for x in loader:
            _ = model(x)

    end = time.time()

    cpu_time = end - start
    print("CPU time (s):", cpu_time)
    return cpu_time


### import the ResNet50 model with default (best availabe) weights 

In [52]:
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)

### weights come with the preprocessing pipeline matching the characteristics of the training data:

In [53]:
preprocess = weights.transforms()

### here comes the fun: 

first we should clone the repo so we can load data to colab from VS code: 

In [54]:
! git clone https://github.com/zakariaaithssain/gpu-acceleration-n8n-pipeline.git

fatal: destination path 'gpu-acceleration-n8n-pipeline' already exists and is not an empty directory.


In [69]:
! ls gpu-acceleration-n8n-pipeline/notebooks


overall_exploration.ipynb  p2_gpu_acceleration.ipynb
p1_gpu_acceleration.ipynb  p3_gpu_acceleration.ipynb


### we set all the acceleration methods we discussed in the `overall_exploration` notebook, and then compare it to the CPU default behavior. 
**Note:** we set a high batch size because GPU excels more with them. 

In [68]:
dataset = InferenceDataset(images_dir=SAMPLE_DIR, transform=preprocess)
loader = data_loader(dataset, batch_size=100, num_workers=2, pin_memory=True)

FileNotFoundError: [Errno 2] No such file or directory: '/content/gpu-acceleration-n8n-pipeline/notebooks/imagenet-sample-images'

#### now we launch inference test for both devices: 

In [ ]:
cpu_time = cpu_inference(model, loader)
#non blocking is active so we use every possible acceleration method
gpu_time = gpu_inference(model, loader, non_blocking=True)

#### we use the following formula to calculate the graphical acceleration: 
**SPEED UP = CPU TIME / GPU TIME**

In [ ]:
speed_up = cpu_time/gpu_time 
print("SPEED UP: ", speed_up)